<a href="https://colab.research.google.com/github/marcomag416/MLDL/blob/main/domain_adaptation_4_colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary code
Feel free to delete/skip this part if run in a persistent environment

In [1]:
import os
import pandas as pd
from PIL import Image
import sys
import requests
from zipfile import ZipFile
from io import BytesIO

In [2]:
#install and import wandb for data collecting
!pip install wandb
import wandb

wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
#download and extract project repository

url= "https://github.com/marcomag416/MLDL/archive/refs/heads/main.zip"

# Send a GET request to the URL
response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./')
    print('Download and extraction complete!')

sys.path.insert(0, './MLDL-main')

Download and extraction complete!


In [4]:
#download cityscapes dataset
from google.colab import drive
drive.mount('/content/drive')

cityscape_dataset_path = "./dataset/Cityscapes/Cityspaces"

#extract zip file
if not os.path.exists(cityscape_dataset_path):
  print("Extracting dataset...")
  with ZipFile("/content/drive/MyDrive/Colab Notebooks/dataset/Cityscapes.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset")



Mounted at /content/drive
Extracting dataset...


In [5]:
#download and index gta5 dataset

gta_path = "/content/drive/MyDrive/Colab Notebooks/dataset/GTA5.zip"

# Define paths
gta_dataset_path = "./dataset/"
images_path = gta_dataset_path + 'GTA5/images'
labels_path = gta_dataset_path + 'GTA5/labels'

#extract zip file
if not os.path.exists(images_path):
  print("Extracting dataset...")
  with ZipFile(gta_path, 'r') as zip_ref:
    zip_ref.extractall(gta_dataset_path)


# Initialize lists to hold data
data = []

# Load images and corresponding masks
for image_filename in os.listdir(images_path):
    if image_filename.endswith('.png'):
        image_path = os.path.join(images_path, image_filename)
        mask_path = os.path.join(labels_path, image_filename)

        # Check if corresponding mask file exists
        if os.path.exists(mask_path):
            # Open image and mask to ensure they can be loaded (optional, for validation)
            try:
                image = Image.open(image_path)
                mask = Image.open(mask_path)

                # Add data to list
                data.append({
                    'image_path': image_path,
                    'mask_path': mask_path
                })
            except Exception as e:
                print(f"Error loading {image_path} or {mask_path}: {e}")

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('./gta5_segmentation_dataset.csv', index=False)

print("Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'")

Extracting dataset...
Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'


# 4 Adversarial domain adaptation

## Datasets

In [6]:
#import from packages
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn
from torch.optim.lr_scheduler import PolynomialLR
from timeit import default_timer as timer
from torch.autograd import Variable

#other imports
from pytorchdl_gta5.labels import GTA5Labels_TaskCV2017
from models.bisenet.build_bisenet import BiSeNet
from models.domainAdaptationModule import DomainAdaptationModule

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)


In [7]:
# Define the custom dataset class
class GTASegmentationDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = self._create_label_mapping()
        #self.color_jitter = T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
        #self.gaussian_blur = T.GaussianBlur(kernel_size=(3, 7), sigma=(0.1, 5))


    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.data_frame.iloc[idx, 0]
        mask_name = self.data_frame.iloc[idx, 1]

        image = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('RGB')

        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image, mask = augmented['image'], augmented['mask']

        #image = self.color_jitter(image)
        #image = self.gaussian_blur(image)

        mask = self._map_mask(np.array(mask))

        if self.transform:
            # Convert mask to tensor without normalization
            mask = torch.from_numpy(mask).permute(2, 0, 1).float()
            mask = mask[0]

        return image, mask

    def _create_label_mapping(self):
        label_mapping = {label.color: label.ID for label in GTA5Labels_TaskCV2017.list_}
        label_mapping[(0, 0, 0)] = 255  # Ensure unmapped colors go to 'unlabeled'
        return label_mapping

    def _map_mask(self, mask):
        new_mask = np.zeros_like(mask)
        for color, label_id in self.label_mapping.items():
            color_mask = np.all(mask == color, axis=-1)
            new_mask[color_mask] = label_id  # Use label_id instead of color
        return new_mask



# Define paths
csv_file = './gta5_segmentation_dataset.csv'

# Define image transformations
transform = A.Compose([
    A.Resize(720, 1280),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    #A.RandomRotate90(),
    A.HorizontalFlip(p=0.5),
    #A.VerticalFlip(p=0.5),
    A.RandomResizedCrop(height=720, width=1280, scale=(0.8, 1.0)),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.GaussianBlur((3, 7), sigma_limit=(0.1, 5)),
    ToTensorV2()
])

# Create the dataset and dataloader
gta_dataset = GTASegmentationDataset(csv_file=csv_file, transform=transform)

print(f"GTA_Dataset size: {len(gta_dataset)}")

GTA_Dataset size: 2500


In [8]:
class Cityscapes(Dataset):
    def __init__(self, root_dir, split, transforms=None, label_type='gtFine_labelTrainIds'):
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.label_type = label_type

        self.images_dir = f"{root_dir}/images/{split}"
        self.labels_dir = f"{root_dir}/gtFine/{split}"

        self.image_paths = []
        self.label_paths = []

        # Manually iterate over directories
        cities = [city for city in os.listdir(self.images_dir) if os.path.isdir(f"{self.images_dir}/{city}")]
        for city in cities:
            img_dir_city = f"{self.images_dir}/{city}"
            lbl_dir_city = f"{self.labels_dir}/{city}"

            if not os.path.isdir(img_dir_city) or not os.path.isdir(lbl_dir_city):
                continue

            for img_file in os.listdir(img_dir_city):
                if img_file.endswith('_leftImg8bit.png'):
                    img_path = f"{img_dir_city}/{img_file}"
                    lbl_file = img_file.replace('_leftImg8bit.png', f'_{self.label_type}.png')
                    lbl_path = f"{lbl_dir_city}/{lbl_file}"

                    if os.path.isfile(img_path) and os.path.isfile(lbl_path):
                        self.image_paths.append(img_path)
                        self.label_paths.append(lbl_path)
                    else:
                        print(f"Warning: Image or label file not found for {img_file}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        lbl_path = self.label_paths[idx]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(lbl_path)

        image = np.array(image)
        label = np.array(label)

        if self.transforms:
            augmented = self.transforms(image=image, mask=label)
            image, label = augmented['image'], augmented['mask']

        return image, label


# Example usage
image_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.HorizontalFlip(p=0.5),
    #A.VerticalFlip(p=0.5),
    A.RandomResizedCrop(height=720, width=1280, scale=(0.8, 1.0)),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.GaussianBlur((3, 7), sigma_limit=(0.1, 5)),
    A.Resize(720, 1280),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

city_dataset_val = Cityscapes(root_dir=cityscape_dataset_path, split='val', transforms=image_transforms)
city_dataset_train = Cityscapes(root_dir=cityscape_dataset_path, split='train', transforms=train_transforms)


print(f"Cityscapes_Dataset_val size: {len(city_dataset_val)}")
print(f"Cityscapes_Dataset_train size: {len(city_dataset_train)}")

Cityscapes_Dataset_val size: 500
Cityscapes_Dataset_train size: 1572


## Train and validation functions

In [9]:
def train_step(model, d_model1, d_model2, d_model3,
          optimizer_model, optimizer_d1, optimizer_d2, optimizer_d3,
          src_dataloader, trg_dataloader, loss_seg_fn, loss_bce_fn,
          step_size,
          lambda_seg=[1.0, 1.0, 1.0], lambda_adv=[1.0, 1.0, 1.0]):
    # labels for adversarial training
    source_label = 0
    target_label = 1

    discriminators = [d_model1, d_model2, d_model3]
    disc_optimizers = [optimizer_d1, optimizer_d2, optimizer_d3]

    # set the models and discriminators to training mode
    model.train()
    for discr in discriminators:
      discr.train()

    src_data_iter = enumerate(src_dataloader)
    trg_data_iter = enumerate(trg_dataloader)

    G_seg_loss = 0.0
    G_adv_loss = 0.0
    D_loss = 0.0

    # reset optimizer gradients
    optimizer_model.zero_grad()
    for opt in disc_optimizers:
      opt.zero_grad()

    for idx in range(step_size):
        # ---- train G ----

        # disable gradient in discriminators
        for discr in discriminators:
          for param in discr.parameters():
                param.requires_grad = False

        # train with source
        _, (src_input, src_labels) = next(src_data_iter)
        src_input, src_labels = Variable(src_input).to(device), Variable(src_labels.long()).to(device)

        src_preds = model(src_input)

        loss_seg = 0.0
        for lamb, pred in zip(lambda_seg, src_preds):
          loss = loss_seg_fn(pred, src_labels)
          loss_seg += lamb * loss
        loss_seg = loss_seg / step_size

        # train with target
        _, (trg_input, _) = next(src_data_iter)
        trg_input = Variable(trg_input).to(device)

        trg_preds = model(trg_input)

        loss_adv = 0.0
        for discr, pred, lamd in zip(discriminators, trg_preds, lambda_adv):
          D_out = discr(F.softmax(pred))
          loss_d =  loss_bce_fn(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label)).to(device))
          loss_adv += loss_d * lamb
        loss_adv = loss_adv / step_size

        # ---- train D ----

        # enable gradient in discriminators
        for discr in discriminators:
          for param in discr.parameters():
                param.requires_grad = True

        batch_D_loss = 0.0
        for discr, src_pred, trg_pred in zip(discriminators, src_preds, trg_preds):
         src_pred.detach()
         trg_pred.detach()
         tot_pred = torch.cat((src_pred, trg_pred), dim=0) #concatenate along batch dimension
         D_out = discr(F.softmax(tot_pred))

         src_shape = list(D_out.size())
         src_shape[0] = src_pred.size()[0]
         trg_shape = list(D_out.size())
         trg_shape[0] = trg_pred.size()[0]

         src_label =  Variable(torch.FloatTensor(size=src_shape).fill_(source_label)).to(device)
         trg_label =  Variable(torch.FloatTensor(size=trg_shape).fill_(target_label)).to(device)
         tot_label = torch.cat((src_label, trg_label), dim=0)

         loss_d =  loss_bce_fn(D_out, tot_label)
         batch_D_loss += loss_d / step_size

        """
        # train with source
        for pred in src_preds:
          pred.detach()
          print(pred.shape)

        losses = [loss_seg, loss_adv]
        for discr, pred in zip(discriminators, src_preds):
          D_out = discr(F.softmax(pred))
          print(D_out.shape)
          loss_d =  loss_bce_fn(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label)).to(device))
          loss_d = loss_d / epoch_size / 2
          losses.append(loss_d)
          batch_D_loss += loss_d

        # train with target
        for pred in trg_preds:
          pred.detach()
          print(pred.shape)

        for discr, pred in zip(discriminators, trg_preds):
          pred.detach()
          D_out = discr(F.softmax(pred))
          print(D_out.shape)
          loss_d =  loss_bce_fn(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(target_label)).to(device))
          loss_d = loss_d / epoch_size / 2
          losses.append(loss_d)
          batch_D_loss += loss_d
        """

        torch.autograd.backward([loss_seg, loss_adv, batch_D_loss])

        #wandb.log({"train G/Batch segmentation loss": loss_seg, "train G/Batch adversarial loss": loss_adv, "train D/Batch loss": batch_D_loss})
        D_loss += batch_D_loss
        G_adv_loss += loss_adv
        G_seg_loss += loss_seg

    #optimizers steps
    optimizer_model.step()
    for opt in disc_optimizers:
      opt.step()

    wandb.log({"train G/Step segmentation loss": G_seg_loss, "train G/Step adversarial loss": G_adv_loss, "train D/Step loss": D_loss}, commit=False)

    return


def compute_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds[target_inds]).sum().item()
        union = pred_inds.sum().item() + target_inds.sum().item() - intersection
        if union == 0:
            ious.append(float('nan'))  # If there is no union, set IoU to NaN
        else:
            ious.append(intersection / union)

    return np.array(ious)

def eval(model, dataloader, loss_fn, device, num_classes=19):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total = 0
    all_ious = []  # List to store IoUs for each batch

    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device, dtype=torch.long)

            outputs_test = model(inputs_test)  # Forward pass
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)

            # Compute IoU for this batch
            batch_ious = compute_iou(predicted_test, targets_test, num_classes)
            all_ious.append(batch_ious)

    # Calculate average loss
    avg_loss = test_loss / total

    # Calculate mean IoU
    all_ious = np.array(all_ious)
    mean_iou = np.nanmean(all_ious, axis=0)  # Mean IoU for each class
    miou = np.nanmean(mean_iou)  # Mean IoU across all classes

    wandb.log({"val/Validation loss": avg_loss, "val/mIoU": miou}, commit=False)

    return avg_loss, miou, mean_iou

## Experiment set-up

In [10]:
# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

#save hyperparameters
config = {
    "learning_rate_G": 2.5e-2,
    "momentum_G": 0.9,
    "weight_decay_G": 1e-4,
    "learning_rate_D": 2.5e-4,
    "num_steps": int(1e4),
    "step_size": 4,
    "batch_size": 2,
    "validation_period": 200,
    "lambda_seg": [1.0, 0.1, 0.01],
    "lambda_adv": [1e-3, 2e-4, 4e-5],
    "target dataset": "Cityscapes",
    "source dataset": "GTA",
    "scheduler": "None",
    "scheduler_poly_power": 0.9,
    "optimizer D": "Adam",
    "optimizer G": "SGD",
    "transformations": "Gaussian blur, random crop, horFlip, colorJitter",
    "pretrained weights": True
}


# create dataloaders
gta_dataloader = DataLoader(gta_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4)
city_dataloader_val = DataLoader(city_dataset_val, batch_size=config["batch_size"], shuffle=False)
city_dataloader_train = DataLoader(city_dataset_train, batch_size=config["batch_size"], shuffle=True, num_workers=4)

# Initialize the model
model = BiSeNet(num_classes=19, context_path=context_path).to(device)
d_model1 = DomainAdaptationModule(num_classes = 19).to(device)
d_model2 = DomainAdaptationModule(num_classes = 19).to(device)
d_model3 = DomainAdaptationModule(num_classes = 19).to(device)

seg_loss_fn = nn.CrossEntropyLoss(ignore_index=255)
bce_loss_fn = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=config["learning_rate_G"], momentum=config["momentum_G"], weight_decay=config["weight_decay_G"])
#optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate_G"])
optimizer_d1 = torch.optim.Adam(d_model1.parameters(), lr=config["learning_rate_D"], betas=(0.9, 0.99))
optimizer_d2 = torch.optim.Adam(d_model2.parameters(), lr=config["learning_rate_D"], betas=(0.9, 0.99))
optimizer_d3 = torch.optim.Adam(d_model3.parameters(), lr=config["learning_rate_D"], betas=(0.9, 0.99))

#scheduler_g = PolynomialLR(optimizer, total_iters=config["num_steps"], power=config["scheduler_poly_power"])
#scheduler_d1 = PolynomialLR(optimizer_d1, total_iters=config["num_steps"], power=config["scheduler_poly_power"])
#scheduler_d2 = PolynomialLR(optimizer_d2, total_iters=config["num_steps"], power=config["scheduler_poly_power"])
#scheduler_d3 = PolynomialLR(optimizer_d3, total_iters=config["num_steps"], power=config["scheduler_poly_power"])


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 164MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 105MB/s]


In [11]:
#run names
project_name = "domain_adaptation_right"
run_name = "pretrained19_no_scheduler"
run_id = ""

#set epoch to 0 to start training from scratch
load_step = 0

#path for loading/saving model weight
weight_dir = f"./drive/MyDrive/Colab Notebooks/model_weights/{project_name}"
load_weight_path = weight_dir + f"/{run_name}_step{load_step}.pth"

resume = "never"
id = None
if(load_step > 0):
  resume="must"
  model.load_state_dict(torch.load(load_weight_path))
  #optimizer.load_state_dict(torch.load(state_path))
  id = run_id
  load_step += 1 #start training from next step

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [12]:
pretrained_weights_path = "./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/gaussian_jitter_epoch19.pth"
if config["pretrained weights"] and pretrained_weights_path:
  model.load_state_dict(torch.load(pretrained_weights_path))

## Run experiment

In [ ]:
#create path for model weights
if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)

#start wandb run
wandb.init(id=id, project=project_name, name=run_name, config=config, resume=resume)

best_mIoU = 0.0

start_time = timer()

# Setup training and save the results
for step in range(load_step, config["num_steps"]):
    wandb.log({"Step": step}, commit=False)
    train_step(model, d_model1, d_model2, d_model3, optimizer_model=optimizer, optimizer_d1=optimizer_d1, optimizer_d2=optimizer_d2, optimizer_d3=optimizer_d3, src_dataloader=gta_dataloader, trg_dataloader=city_dataloader_val, loss_seg_fn=seg_loss_fn, loss_bce_fn=bce_loss_fn, step_size=config["step_size"], lambda_seg=config["lambda_seg"], lambda_adv=config["lambda_adv"])

    if config["scheduler"] != "None":
      scheduler_g.step()
      scheduler_d1.step()
      scheduler_d2.step()
      scheduler_d3.step()

    #validate model every validation period
    if((step+1) % config["validation_period"] == 0):
      avg_loss, miou, IoUs = eval(model, city_dataloader_val, seg_loss_fn, device=device)
      print(f"Step: {step}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%")
      print(f"\t IoU per class: {IoUs}")
      if(miou > best_mIoU and (step + 1) > (2* config["validation_period"])):
        file_name = f"/{run_name}_step{step}.pth"
        torch.save(model.state_dict(), weight_dir + file_name)
        best_mIoU = miou
        print("Model weights saved as: "+ file_name)

    wandb.log({}, commit=True)


# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")


wandb: Currently logged in as: marcomag416 (marco-magnanini). Use `wandb login --relogin` to force relogin


In [ ]:
wandb.finish()